Вопрос 3
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
def z_stat(s1, n1, s2, n2):
    p1, p2 = s1 / n1, s2 / n2
    P = (s1 + s2) / (n1 + n2)
    return (p1 - p2) / (np.sqrt(P*(1-P)*(1/n1 + 1/n2)))

In [3]:
z = z_stat(10, 34, 4, 16)

In [4]:
1-stats.norm.cdf(z)

0.37293045872523534

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. 
Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:
логистическая регрессия по признакам X_1,X_2,X_3;
логистическая регрессия по признакам X_4,X_5,X_6X.
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5 * 10^{-8}, нужно ввести 8).

In [5]:
df = pd.read_csv("banknotes.txt", sep = '\t')

In [6]:
df.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [7]:
y = df['real']
X = df.drop('real', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 50, random_state = 1)

In [8]:
part_1 = ['X1', 'X2', 'X3']
part_2 = ['X4', 'X5', 'X6']

In [9]:
y_1 = LogisticRegression(solver='liblinear').fit(X_train[part_1], y_train).predict(X_test[part_1])

In [10]:
y_2 = LogisticRegression(solver='liblinear').fit(X_train[part_2], y_train).predict(X_test[part_2])

In [11]:
err_1 = np.abs(y_test - y_1)

In [12]:
err_2 = np.abs(y_test - y_2)

In [13]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_1))
print(metrics.classification_report(y_test, y_2))

              precision    recall  f1-score   support

           0       0.74      0.87      0.80        23
           1       0.87      0.74      0.80        27

    accuracy                           0.80        50
   macro avg       0.81      0.81      0.80        50
weighted avg       0.81      0.80      0.80        50

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        23
           1       0.96      1.00      0.98        27

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50



In [14]:
m1_er = sum(y_test != y_1)

In [15]:
m2_er = sum(y_test != y_2)

In [16]:
m1_er, m2_er

(10, 1)

In [17]:
s1 = (y_test != y_1).astype(int)
s2 = (y_test != y_2).astype(int)

In [18]:
f = (s1 - s2).value_counts()[1]
g = (s1 - s2).value_counts()[-1]

In [19]:
def z_stat_rel(f, g, n):
    return (f - g) / (np.sqrt(f + g - ((f - g)**2 / n)))

In [20]:
z = z_stat_rel(f, g, 50)

In [21]:
2 * (1 - stats.norm.cdf(z))

0.0032969384555543435


В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [22]:
def confint_rel(f, g, n, alpha):
    z = stats.norm.ppf(1 - alpha / 2)
    base = (f - g) / n
    addition = np.sqrt(((f + g) / n**2) - ((f - g)**2 / n**3))
    return (base - z*addition, base + z*addition)

In [23]:
print("95%%: [%f, %f]" % confint_rel(f, g, 50, 0.05))

95%: [0.059945, 0.300055]


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [24]:
stat = (541.4 - 525) / (100 / np.sqrt(100))
1 - stats.norm.cdf(stat)

0.05050258347410397

In [25]:
stat = (541.5 - 525) / (100 / np.sqrt(100))
1 - stats.norm.cdf(stat)

0.0494714680336481

Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

49, 58, 75, 110, 112, 132, 151, 276, 281, 362

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы H_0\colon med X = 200H 
0
​	
 :medX=200 на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет.

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [26]:
data = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])

In [27]:
stats.wilcoxon(data - 200)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15,

и в 9 лесах, где идёт вырубка:

17, 18, 18, 15, 12, 4, 14, 15, 10.

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [28]:
f_12 = np.array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])
f_9 = np.array([17, 18, 18, 15, 12, 4, 14, 15, 10])

In [29]:
stats.mannwhitneyu(f_12, f_9, alternative = 'greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.
С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
сделайте по 1000 псевдовыборок из каждой выборки.

In [30]:
df = pd.read_csv("challenger.txt", sep = '\t')

In [31]:
df = df.drop(df.columns[0], axis = 1)

In [32]:
df.head()

,Temperature,Incident
0,18.9,0
1,21.1,1
2,20.6,0
3,20.0,0
4,19.4,0


In [33]:
incident = np.array(df.Temperature[df.Incident == 1])
no_incident = np.array(df.Temperature[df.Incident == 0])

In [34]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [35]:
np.random.seed(0)
no_incident_means = list(map(np.mean, get_bootstrap_samples(no_incident, 1000)))
incident_means = list(map(np.mean, get_bootstrap_samples(incident, 1000)))

In [36]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [37]:
delta_means = list(map(lambda x: x[0] - x[1], list(zip(no_incident_means, incident_means))))
intervals = stat_intervals(delta_means,0.05)
intervals

array([1.42299107, 7.93861607])

На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

Чтобы получить такое же значение, как мы:

установите random seed = 0;
возьмите 10000 перестановок.

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [38]:
w_data = pd.read_csv("water.txt", sep = '\t')

In [39]:
w_data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [40]:
w_data['mortality'].corr(w_data['hardness'])

-0.6548486232042464

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [41]:
w_data['mortality'].corr(w_data['hardness'], method = 'spearman')

-0.6316646189166502

Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [42]:
w_data.loc[w_data['location'] == 'South', ['mortality', 'hardness']].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [43]:
w_data.loc[w_data['location'] == 'North', ['mortality', 'hardness']].corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [44]:
def mcc(a, b, c, d):
    return (a * d - b * c) / np.sqrt((a + b) * (a + c) * (b + d) * (c + d))

In [45]:
mcc(203, 718, 239, 515)

-0.10900237458678963

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}
 , нужно ввести 8).

In [46]:
stats.chi2_contingency(np.array([[203, 718], [239, 515]]))

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 677.96597015],
        [198.96597015, 555.03402985]]))

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

Не доволен	Более или менее	Доволен
Не очень счастлив	197	111	33
Достаточно счастлив	382	685	331
Очень счастлив	110	342	333
Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [47]:
happy = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

In [48]:
chi = stats.chi2_contingency(happy)

In [49]:
chi

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

In [50]:
np.sqrt(chi[0]/(np.sum(happy) * 2))

0.2412013934500338

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [51]:
aucs = pd.read_csv("AUCs.txt", sep = '\t')

In [52]:
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [53]:
corr_arr = []
for i, i_col in enumerate(aucs.columns[1:]):
    for j, j_col in enumerate(aucs.columns[1:]):
        if i < j:
            corr, p = stats.wilcoxon(aucs[i_col], aucs[j_col])
            corr_arr.append([i_col, j_col, corr, p])
    

In [54]:
corr_arr

[['C4.5', 'C4.5+m', 6.5, 0.01075713311978963],
 ['C4.5', 'C4.5+cf', 43.0, 0.861262330095348],
 ['C4.5', 'C4.5+m+cf', 11.0, 0.015906444101703374],
 ['C4.5+m', 'C4.5+cf', 17.0, 0.046332729793395394],
 ['C4.5+m', 'C4.5+m+cf', 22.0, 0.3278256758446406],
 ['C4.5+cf', 'C4.5+m+cf', 10.0, 0.022909099354356588]]

In [55]:
corr = pd.DataFrame(corr_arr)
corr.columns = ['X', 'Y', 'corr', 'p']

In [56]:
corr[corr.p == corr.p.min()]

,X,Y,corr,p
0,C4.5,C4.5+m,6.5,0.010757


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [57]:
(corr.p < 0.05).value_counts()

True     4
False    2
Name: p, dtype: int64

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [58]:
from statsmodels.sandbox.stats.multicomp import multipletests
r, p, a1, a2 = multipletests(corr.p, alpha = 0.05, method = 'holm')

In [59]:
corr['p_correct'] = p
corr['reject'] = r

In [60]:
corr

,X,Y,corr,p,p_correct,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


In [61]:
corr.reject.value_counts()

False    6
Name: reject, dtype: int64

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [62]:
r, p, a1, a2 = multipletests(corr.p, alpha = 0.05, method = 'fdr_bh')
corr['p_correct'] = p
corr['reject'] = r

In [63]:
corr

,X,Y,corr,p,p_correct,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True


In [64]:
corr.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
    Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [65]:
data = pd.read_csv("botswana.tsv", sep = '\t')

In [66]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [67]:
data.religion.nunique()

4

In [68]:
sum(np.sum(data.isna(), axis = 1) == 0)

1834

Вопрос 3
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x_1x 
1
​	
  на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак
x_2 =
{1,0,x1='не применимо',иначе;
x 
2
​	
 ={ 
1,
0,
​	
  
x 
1
​	
 =’не применимо’,
иначе;
​	
 
заменить "не применимо" в x_1x 
1
​	
  на произвольную константу cc, которая среди других значений x_1x 
1
​	
  не встречается.
Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,
y=β 
0
​	
 +β 
1
​	
 x 
1
​	
 +β 
2
​	
 x 
2
​	
 ,
на тех объектах, где x_1x 
1
​	
  было измерено, регрессионное уравнение примет вид
y=\beta_0 + \beta_1 x,
y=β 
0
​	
 +β 
1
​	
 x,
а там, где x_1x 
1
​	
  было "не применимо", получится
y=\beta_0 + \beta_1 c + \beta_2.
y=β 
0
​	
 +β 
1
​	
 c+β 
2
​	
 .
Выбор cc влияет только на значение и интерпретацию \beta_2β 
2
​	
 , но не \beta_1β 
1
​	
 .

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XX будет мультиколлинеарность.
Замените NaN в признаке agefm на c_{agefm}=0c 
agefm
​	
 =0.
У объектов, где nevermarr = 1, замените NaN в признаке heduc на c_{heduc_1}=-1c 
heduc 
1
​	
 
​	
 =−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?

In [69]:
data['nevermarr'] = 1 - data['evermarr']
data.drop('evermarr', axis = 1, inplace = True)
data['agefm'].fillna(0, inplace = True)
data.loc[data['nevermarr'] == 1, 'heduc'] = -1
sum(data['heduc'].isna())

123

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (c_{idlnchld}=-1c 
idlnchld
​	
 =−1, c_{heduc_2}=-2c 
heduc 
2
​	
 
​	
 =−2 (значение -1 мы уже использовали), c_{usemeth}=-1c 
usemeth
​	
 =−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [70]:
data['idlnchld_noans'] = data['idlnchld'].apply(lambda x : 1 if np.isnan(x) else 0)
data['heduc_noans'] = data['heduc'].apply(lambda x : 1 if np.isnan(x) else 0)
data['usemeth_noans'] = data['usemeth'].apply(lambda x : 1 if np.isnan(x) else 0)

In [71]:
data.idlnchld.fillna(-1, inplace = True)
data.heduc.fillna(-2, inplace = True)
data.usemeth.fillna(-1, inplace = True)

In [72]:
data = data.dropna()

In [73]:
data.shape[0] * data.shape[1]

78264

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2R 
2
 ? Округлите до трёх знаков после десятичной точки.

In [74]:
import statsmodels.formula.api as smf

In [75]:
model = smf.ols('ceb ~ ' + ' + '.join(data.columns[1:]), data = data)

In [76]:
fit = model.fit()

In [77]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Wed, 26 Aug 2020   Prob (F-statistic):               0.00
Time:                        13:00:52   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans             0.6539      0.153      4.286      0.000       0.355       0.953
heduc_noans               -0.8724      0.145     -6.026      0.000      -1.156      -0.589
usemeth_noans              0.7652      0.196      3.910      0.000       0.382       1.149
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [78]:
data.religion.value_counts()

spirit        1838
other         1076
protestant     989
catholic       445
Name: religion, dtype: int64

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

In [79]:
from statsmodels.stats.diagnostic import het_breuschpagan as bp

In [80]:
bp(fit.resid, fit.model.exog)

(1120.1205909391392,
 1.1452927633433924e-225,
 79.04622432816002,
 2.0791008841120013e-262)

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [81]:
col_del = ['religion', 'radio', 'tv']

In [82]:
data_del = data.drop(col_del, axis = 1)

In [83]:
data_del.head()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1,0,0,0
1,2,43,11,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,0,0,0,0
2,0,49,4,4.0,1.0,0.0,22.0,1.0,1,1.0,0.0,0,0,0,0
3,0,24,12,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1,0,0,0
4,3,32,13,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,0,0,0,0


In [84]:
model_del = smf.ols('ceb ~ ' + ' + '.join(data_del.columns[1:]), data = data_del)

In [85]:
fit_del = model_del.fit()

In [86]:
fit_del.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Wed, 26 Aug 2020   Prob (F-statistic):               0.00
Time:                        13:00:52   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.393      0.000      -1.459      -0.681
age                0.1702      0.003     52.271      0.000       0.164       0.177
educ              -0.0729      0.007    -10.285      0.000      -0.087      -0.059
idlnchld           0.0770      0.011      7.042      0.000       0.056       0.098
knowmeth           0.5610      0.121      4.628      0.000       0.323       0.799
usemeth            0.6516      0.048     13.537      0.000       0.557       0.746
agefm             -0.0606      0.007     -9.240      0.000      -0.073      -0.048
heduc             -0.0573      0.008     -7.186      0.000      -0.073      -0.042
urban             -0.2190      0.047     -4.682      0.000      -0.311      -0.127
electric          -0.3207      0.070     -4.584      0.000      -0.458      -0.184
bicycle            0.2046      0.049      4.154      0.000       0.108       0.301
nevermarr         -2.2501      0.148    -15.231      0.000      -2.540      -1.961
idlnchld_noans     0.6565      0.152      4.310      0.000       0.358       0.955
heduc_noans       -0.8853      0.145     -6.122      0.000      -1.169      -0.602
usemeth_noans      0.7732      0.196      3.955      0.000       0.390       1.156
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
bp(fit_del.resid, fit_del.model.exog)

(1112.4700385717542,
 1.1197458896534061e-228,
 106.41517187062946,
 3.592611312534989e-265)

In [88]:
fit.compare_f_test(fit_del)

(0.9192357784629903, 0.4672305547275616, 5.0)

Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением c_{usemeth }=-1c 
usemeth
​	
 =−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5\times10^{-8}5.5×10 
−8
 , нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [89]:
col_meth = ['usemeth_noans', 'usemeth']
data_del_meth = data_del.drop(col_meth, axis = 1)

In [91]:
data_del_meth.head()

,ceb,age,educ,idlnchld,knowmeth,agefm,heduc,urban,electric,bicycle,nevermarr,idlnchld_noans,heduc_noans
0,0,18,10,4.0,1.0,0.0,-1.0,1,1.0,1.0,1,0,0
1,2,43,11,2.0,1.0,20.0,14.0,1,1.0,1.0,0,0,0
2,0,49,4,4.0,1.0,22.0,1.0,1,1.0,0.0,0,0,0
3,0,24,12,2.0,1.0,0.0,-1.0,1,1.0,1.0,1,0,0
4,3,32,13,3.0,1.0,24.0,12.0,1,1.0,1.0,0,0,0


In [93]:
model_del_meth = smf.ols('ceb ~ ' + ' + '.join(data_del_meth.columns[1:]), data = data_del_meth)

In [94]:
fit_del_meth = model_del_meth.fit()

In [95]:
fit_del_meth.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Wed, 26 Aug 2020   Prob (F-statistic):               0.00
Time:                        13:08:33   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.897      0.000      -1.590      -0.796
age                0.1776      0.003     54.368      0.000       0.171       0.184
educ              -0.0560      0.007     -7.862      0.000      -0.070      -0.042
idlnchld           0.0705      0.011      6.317      0.000       0.049       0.092
knowmeth           0.8739      0.121      7.203      0.000       0.636       1.112
agefm             -0.0649      0.007     -9.716      0.000      -0.078      -0.052
heduc             -0.0521      0.008     -6.411      0.000      -0.068      -0.036
urban             -0.1866      0.048     -3.917      0.000      -0.280      -0.093
electric          -0.3218      0.071     -4.505      0.000      -0.462      -0.182
bicycle            0.1979      0.050      3.935      0.000       0.099       0.296
nevermarr         -2.3625      0.150    -15.707      0.000      -2.657      -2.068
idlnchld_noans     0.5266      0.155      3.393      0.001       0.222       0.831
heduc_noans       -0.7947      0.147     -5.389      0.000      -1.084      -0.506
==============================================================================
Omnibus:                      250.641   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              936.515
Skew:                          -0.158   Prob(JB):                    4.35e-204
Kurtosis:                       5.251   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [96]:
fit_del.compare_f_test(fit_del_meth)

(92.89058230109758, 3.1552009480386492e-40, 2.0)